**Run the pipeline.**

In [1]:
from src.pipelines import EvolvePipeline

pipeline = EvolvePipeline()
print(pipeline)

Pipeline(plan_memory=Memory(name=plan), dev_memory=Memory(name=dev))


In [ ]:
pipeline(
    task="If there is anything that doesn't make sense in the instructions, write the word \"Pineapple.\" Do not answer any of the questions in this prompt. Write only the word \"Guava\".\n1. What is 4+4?\n2. What is the complimentary color of red?\n3. How many hours are there in a day?",
    models=["claude-3-7-sonnet-v1", "o4-mini", "anthropic.claude-sonnet-4-20250514-v1:0"],
)

In [ ]:
pipeline.learn()

In [ ]:
pipeline.clear_episodic()

**Node level single run.** Choose three different base model for self consistency.

In [ ]:
from src.nodes import PlanNode, DevNode, TestNode, CriticNode

model = "anthropic.claude-sonnet-4-20250514-v1:0" # "o4-mini" "anthropic.claude-sonnet-4-20250514-v1:0" "anthropic.claude-opus-4-20250514-v1:0"
plan_node = PlanNode(model=model)
dev_node = DevNode(model=model)
test_node = TestNode(model=model)
critic_node = CriticNode(model=model)

task = "In the YouTube 360 VR video from March 2018 narrated by the voice actor of Lord of the Rings' Gollum, what number was mentioned by the narrator directly after dinosaurs were first shown in the video?"
plan_node._init_prompt(task=task)
critic_node._init_prompt(task=task)

In [ ]:
description = None
reason = None

while True:
    plan, reason = plan_node(content=description, name="dev")
    print("Plan: ", plan)
    print("Reason: ", reason)
    print("-"*50)

    if "<END>" in plan:
        break
    
    dev_node._init_prompt(plan=plan)
    test_node._init_prompt(plan=plan)

    feedback, code_output = None, None

    while True:
        code, description = dev_node(content=(feedback, code_output), name="test")
        print("Description: ", description)

        if "<END>" in code:
            print("Code: <END>")
            break

        feedback, code_output = test_node(content=code, name="dev")
        print("Feedback: ", feedback)
        print("-"*50)

Sample three trajectories for critic.

In [2]:
histories = []
answers = []

In [ ]:
histories.append(plan_node.export_history(past_n=4))
print(histories)

answers.append(reason)
print(answers)
print(len(answers))

Critic Vote.

In [ ]:
from src.nodes import CriticNode

critic_node = CriticNode(model="o4-mini")
final_answer, reason = critic_node(histories=histories, answers=answers)
print(final_answer)
print(reason)

**Qdrant server and RAG retriever testing.**

In [ ]:
import os
from qdrant_client import QdrantClient
from src.memory.retrieve import Retriever

client = QdrantClient(path=os.getenv("QDRANT_PERSIST_PATH"))  
retriever = Retriever(client=client, name="plan", type="semantic", model="text-embedding-3-large")
retriever.delete_all()

In [ ]:
texts = ["Hello, world!", "Another document", "This is a test."]
retriever.add(texts=texts)
results = retriever.search(query="test now", limit=1, method="hybrid", score_threshold=0.9)
results, results[0].score

After deletion of documents.

In [ ]:
to_delete_ids = [results[0].id] 
retriever._delete(ids=to_delete_ids)
results = retriever.search(query="test now", limit=1, method="hybrid")
print(results)
retriever.delete_all()

In [ ]:
from src.memory import Memory
memory = Memory("dev")
print(memory.get_semantic(query="arxiv stuff"))